In [ ]:
pip install backtrader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 6.3 MB/s eta 0:00:00


In [ ]:
#기본 실행
import backtrader as bt

if __name__ == '__main__':
  cerebro = bt.Cerebro()
  print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

  cerebro.run()
  print('Final Porfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
Final Porfolio Value: 10000.00


In [ ]:
#현금 세팅
import backtrader as bt

if __name__ == '__main__':
     cerebro = bt.Cerebro()
     cerebro.broker.setcash(100000.0)

     print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

     cerebro.run()

     print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
Final Portfolio Value: 100000.00


In [ ]:
#주식 데이터 추가(MSFT 2011.1.1 ~ 2013.12.31)
from datetime import datetime
import backtrader as bt
import yfinance as yf

if __name__ == '__main__':
    cerebro = bt.Cerebro()
    cerebro.broker.setcash(100000.0)

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # 데이터 피
    data = bt.feeds.PandasData(dataname=yf.download('MSFT',
                                 '2011-01-01', '2013-12-31'))

    cerebro.adddata(data)  # 데이터 피드 추가

    cerebro.run()

    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00


[*********************100%***********************]  1 of 1 completed


Final Portfolio Value: 100000.00


In [ ]:
# SMA 전략
from datetime import datetime
import backtrader as bt
import math
import yfinance as yf


class SmaCross(bt.Strategy):
    params = dict(
        pfast=30,   # 단기 SMA
        pslow=200   # 장기 SMA
    )

    def __init__(self):
        self.sma1 = bt.ind.SMA(period=self.p.pfast)  # 단기 SMA
        self.sma2 = bt.ind.SMA(period=self.p.pslow)  # 장기 SMA

    def next(self):
        if not self.position:  # 포지션이 없을 경우
            # 단기 SMA가 장기 SMA를 위로 돌파했을 경우
            if self.sma1[0] >= self.sma2[0] and self.sma1[-1] < self.sma2[-1]:
                # 진입 가능 포지션 계산 (현금 / 현재 가격)
                order_size = math.floor(self.broker.get_value() / self.datas[0].close)
                self.buy(size=order_size)  # 롱 포지션 진입
        # 장기 SMA가 단기 SMA를 아래로 돌파했을 경우
        elif self.sma1[0] <= self.sma2[0] and self.sma1[-1] > self.sma2[-1]:
            self.close()  # 포지션 종료

if __name__ == '__main__':
    cerebro = bt.Cerebro()
    cerebro.broker.setcash(10000.0)

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # 데이터 피드 생성
    data = bt.feeds.PandasData(dataname=yf.download('MSFT', '2011-01-01', '2018-12-31'))


    cerebro.adddata(data)  # 데이터 피드 추가
    cerebro.addstrategy(SmaCross)  # 전략 추가

    cerebro.run()


    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

    cerebro.plot()

[*********************100%***********************]  1 of 1 completed

Starting Portfolio Value: 10000.00


Final Portfolio Value: 18500.44


In [ ]:
pip install quantstats

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.0 MB/s eta 0:00:00


In [16]:
from datetime import datetime
import backtrader as bt
import math
import quantstats as qs
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt



# SMA 전략
class SmaCross(bt.Strategy):
    params = dict(
        pfast=30,   # 단기 SMA
        pslow=200   # 장기 SMA
    )

    def __init__(self):
        self.sma1 = bt.ind.SMA(period=self.p.pfast)  # 단기 SMA
        self.sma2 = bt.ind.SMA(period=self.p.pslow)  # 장기 SMA

    def next(self):
        if not self.position:  # 포지션이 없을 경우
            # 단기 SMA가 장기 SMA를 위로 돌파했을 경우
            if self.sma1[0] >= self.sma2[0] and self.sma1[-1] < self.sma2[-1]:
                # 진입 가능 포지션 계산 (현금 / 현재 가격)
                order_size = math.floor(self.broker.get_value() / self.datas[0].close)
                self.buy(size=order_size)  # 롱 포지션 진입
        # 장기 SMA가 단기 SMA를 아래로 돌파했을 경우
        elif self.sma1[0] <= self.sma2[0] and self.sma1[-1] > self.sma2[-1]:
            self.close()  # 포지션 종료

if __name__ == '__main__':
    cerebro = bt.Cerebro()
    cerebro.broker.setcash(10000.0)

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # 데이터 피드 생성
    data = bt.feeds.PandasData(dataname=yf.download('MSFT', '2011-01-01', '2018-12-31'))

    cerebro.adddata(data)  # 데이터 피드 추가
    cerebro.addstrategy(SmaCross)  # 전략 추가
    cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')  # 결과 분석 추가

    cerebro.run()

    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

    results = cerebro.run()
    strat = results[0]
    pyfoliozer = strat.analyzers.getbyname('pyfolio')

    returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()
    returns.index = returns.index.tz_convert(None)



    # 간단한 결과 출력
    print(f'\n')
    print("Result:")
    cagr = qs.stats.cagr(returns)
    mdd = qs.stats.max_drawdown(returns)
    sharpe = qs.stats.sharpe(returns)
    print(f"SHARPE: {sharpe:.2f}")
    print(f"CAGR: {cagr * 100:.2f} %")
    print(f"MDD : {mdd * 100:.2f} %")

    # 벤치마크 데이터 생성 (예: S&P 500 지수)
    benchmark_data = yf.download('^GSPC', start='2011-01-01', end='2018-12-31')['Adj Close']
    # 자세한 결과 html 파일로 저장
    qs.reports.html(returns, benchmark=benchmark_data, output=f'SMA_MSFT.html', title='SMA MSFT Strategy Results')



[*********************100%***********************]  1 of 1 completed

Starting Portfolio Value: 10000.00


Final Portfolio Value: 18500.44


[*********************100%***********************]  1 of 1 completed



Result:
SHARPE: 0.55
CAGR: 5.46 %
MDD : -23.52 %



/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwarg

In [17]:
from google.colab import files
files.download('SMA_MSFT.html')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>